In [ ]:
#This was an example challenge to get familiar with making bootstrapping samples
#Bootstrapping samples is a random sample of the original data
#data points may be duplicated

In [ ]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib notebook

In [ ]:
# Let's generate 1000 events drawn from a Gaussian distribution and calculate the mean and std dev

npts = 1000
mu = 10.0
sigma = 1.0

data = np.random.normal(mu,sigma,npts)

print "calculated mean   : %f" % (np.mean(data))
print "calculated std dev: %f" % (np.std(data))


In [ ]:
plt.figure(figsize=(4,4))
plt.hist(data,bins=50);

In [ ]:
# Now, generate a "fake" dataset of 1000 points randomly drawn from data. 
# Because you are pulling out 1000 points (the same number as in data), you will have some
# repeating data points. That's OK!

fake_data=np.random.choice(data,1000)

# When you make this fake dataset, calculate the mean

print np.mean(fake_data)
# YOUR CODE HERE

In [ ]:
# Now do this 1000 times, and keep track of all the different means from these mock datasets. 
# Histogram those means. 
plt.figure()
means=[]
stds=[]
data=np.array(data)
for x in range(1000):
    fake_data=np.random.choice(data,1000)
    stds.append(np.std(fake_data))
    means.append(np.mean(fake_data))
plt.hist(means, bins=50)
print 'average mean: ', np.mean(means)
print 'avergae standard deviation:', np.mean(stds)
# YOUR CODE HERE

In [ ]:
# What is the mean of those distributions?

# What are the values around the mean that enclose 68% of these mock data means? 95%?
'''
diff= abs(np.mean(stds)-stds)
diff.sort()
diff[int(1000*.68)]
print np.mean(stds)-diff[int(1000*.68)]
print np.mean(stds)+diff[int(1000*.68)]
'''
diff1= np.mean(stds)-stds
diff1.sort()
print diff1[(int(1000*.34)-1)]+np.mean(stds)
#print diff1
print diff1[(int(1000*.84)-1)] +np.mean(stds)
print 'mean of stds:',np.mean(stds)
print 'std of stds',np.std(stds)



In [ ]:


def bootstrapping(data):
    npts = len(data)
    indices = np.random.randint(0,npts,npts)

    bs_data= np.array([data[indices].copy()])
        
    return bs_data

# A product of two Gaussians
def signal_2D(npts,means,sigmas):
    pts = []
    for m,s in zip(means,sigmas):
        pts.append(np.random.normal(m,s,npts))
    pts = np.array(pts)
    return pts

In [ ]:
nMC_sig=1000
sigmeans=[5.0,7.0]
sigwidths=[0.1,0.1]
nsig_iteration=200

sig_mean=10.4
sig_width=.06
signal = np.random.normal(sig_mean,sig_width,nsig_iteration)
signal_points= signal_2D(nMC_sig,sigmeans,sigwidths)

print len(signal)
bootstrapping(np.array(signal))

In [ ]:
def calc_pull_1D_GPU_bootstrapping(iterations, nsig, nbkg, MC_sig, MC_bkg, num_bootstrapping_samples, nneigh, sig_width,tag):
    outfile_name='frac_values_%s_sig%d_bkg%d_MCsig%d_MCbkg%d_bs%d_nn%d.dat'%(tag,nsig,nbkg,MC_sig,MC_bkg,num_bootstrapping_samples,nneigh)
    outfile=open(outfile_name,'w')
    print 'writing out to file %s' %outfile_name
    pull_frac_list=[]
    average_best_frac = 0
    frac = []
    fit_frac = []
    fit_frac_uncert = []
    my_gpu = numba.cuda.get_current_device()
    thread_ct = my_gpu.WARP_SIZE


    for num in range(iterations):
        nsig_iteration = np.random.poisson(nsig)
        nbkg_iteration = np.random.poisson(nbkg)
        sig_mean=10.4
        sig_width=.06
        signal = np.random.normal(sig_mean,sig_width,nsig_iteration)

        background = 9.0+(2*np.random.random(nbkg_iteration))
        data = signal.copy()
        data = np.append(data,background.copy())
        signal_compare = np.random.normal(sig_mean,sig_width,MC_sig)
        
        background_compare= 9.0+(2*np.random.random(MC_bkg))
        
        
        block_ct_sig = int(math.ceil(float(MC_sig*(nsig_iteration+nbkg_iteration)) / thread_ct))
        block_ct_bkg = int(math.ceil(float(MC_bkg*(nsig_iteration+nbkg_iteration)) / thread_ct))
        signal_distances_GPU = np.zeros((nsig_iteration+nbkg_iteration)*MC_sig, dtype = np.float32)
        background_distances_GPU = np.zeros((nsig_iteration+nbkg_iteration)*MC_bkg, dtype = np.float32)

        signal_distances=distances_GPU_1D[block_ct_sig, thread_ct](np.float32(data), len(data), np.float32(signal_compare),len(signal_compare), signal_distances_GPU)
        background_distances=distances_GPU_1D[block_ct_bkg, thread_ct](np.float32(data), len(data), np.float32(background_compare), len(background_compare), background_distances_GPU)
        
        signal_prob=sort_GPU_1D((nsig_iteration+nbkg_iteration),MC_sig,np.abs(signal_distances_GPU),nneigh)
        background_prob=sort_GPU_1D((nsig_iteration+nbkg_iteration),MC_bkg,np.abs(background_distances_GPU), nneigh)
        signal_MC_bs = []
        background_MC_bs = []
      
        

        signal_prob_bs = []
        background_prob_bs = []
        
        for i in range(0,num_bootstrapping_samples):
            
            #Generating bootstrapping samples
            signal_MC_bs.append(bootstrapping(signal))
            background_MC_bs.append(bootstrapping(background))
            
            signal_distances_GPU = np.zeros((nsig_iteration+nbkg_iteration)*MC_sig, dtype = np.float32)
            background_distances_GPU = np.zeros((nsig_iteration+nbkg_iteration)*MC_bkg, dtype = np.float32)


            #calculating distances for each point
            signal_distances=distances_GPU_1D[block_ct_sig, thread_ct](np.float32(data), len(data), np.float32(signal_compare),len(signal_compare), signal_distances_GPU)
            background_distances=distances_GPU_1D[block_ct_bkg, thread_ct](np.float32(data), len(data), np.float32(background_compare), len(background_compare), background_distances_GPU)
        

            
            signal_prob_bs.append(sort_GPU((nsig_iteration+nbkg_iteration),MC_sig,  np.abs(signal_distances_GPU),nneigh))
            background_prob_bs.append(sort_GPU((nsig_iteration+nbkg_iteration),MC_bkg, np.abs(background_distances_GPU),nneigh))
            
        
       
        
        
        def tot_prob(frac,sig,bkg):
            tot_prob = frac*sig/MC_sig + ((1-frac)*bkg/MC_bkg)            
            return tot_prob
        
        def probability(frac):
            prob= tot_prob(frac,signal_prob,background_prob)
            nll= -np.log(prob[prob>0]).sum()
            for i in range(0,num_bootstrapping_samples):
                prob = tot_prob(frac,signal_prob_bs[i],background_prob_bs[i])
                nll +=  -np.log(prob[prob>0]).sum()
            return nll
            #return -np.log(tot_prob).sum()
        
        m=Minuit(probability, frac= 0.20, limit_frac=(0.001,1), error_frac=0.001,  errordef = (num_bootstrapping_samples+1)*0.5, print_level=0)
        m.migrad()
        if (m.get_fmin().is_valid):
            param=m.values
            err=m.errors
            fit_frac.append(param["frac"])
            fit_frac_uncert.append(err["frac"])
            pull_frac=((float(nsig_iteration)/(float(nbkg_iteration)+float(nsig_iteration)))-param["frac"])/err["frac"]
            ndata = len(data)
            frac_org = nsig/float(nsig + nbkg)
            nsig_org = frac_org * ndata
            nsig_fit = param["frac"]
            nsig_err = err["frac"]
            pull_frac_list.append(pull_frac)
            output="%f %f %f %d %d %d %d %d %d %d %d\n" % (frac_org,param["frac"],err["frac"], nsig,nsig_iteration,nbkg,nbkg_iteration,MC_sig,MC_bkg,num_bootstrapping_samples,nneigh)
            outfile.write(output)
    outfile.close()
    return pull_frac_list, frac, fit_frac, fit_frac_uncert,iterations,outfile_name





In [ ]:
#Calulates the pulls for the mean and std.
def calc_pull_w_bootstrapping_GPU(pull_iterations, nsig, nbkg,nMC_sig, nMC_bkg, num_bootstrapping_samples,  nneigh,sigwidths, tag="default"):
    outfile_name='frac_values_%s_sig%d_bkg%d_MCsig%d_MCbkg%d_bs%d_nn%d.dat'%(tag,nsig,nbkg,nMC_sig,nMC_bkg,num_bootstrapping_samples,nneigh)
    outfile=open(outfile_name,'w')
    print 'writing out to file %s' %outfile_name
    
    pull_frac_list=[]
    average_best_frac = 0
    frac = []
    fit_frac = []
    fit_frac_uncert = []
    frac_org = nsig/float(nsig+nbkg)
    my_gpu = numba.cuda.get_current_device()
    thread_ct = my_gpu.WARP_SIZE

    for num in range(pull_iterations):        
        # Generate the data for this pull iteration
        nsig_iteration = np.random.poisson(nsig)
        nbkg_iteration = np.random.poisson(nbkg)
        data = gen_sig_and_bkg([nsig_iteration,nbkg_iteration],sigmeans,sigwidths,bkglos,bkghis)

        # Record the original amount of signal and background data
        #frac_iteration = float(nsig_iteration)/(float(nbkg_iteration+nsig_iteration))
        #frac.append(frac_iteration)
                
        # Generate the MC we will use to try to fit the data we just generated!
        signal_points= signal_2D(nMC_sig,sigmeans,sigwidths)
        background_points = background_2D(nMC_bkg,bkglos,bkghis)

        #Block count
        block_ct_sig = int(math.ceil(float(nMC_sig*(nsig_iteration+nbkg_iteration)) / thread_ct))
        block_ct_bkg = int(math.ceil(float(nMC_bkg*(nsig_iteration+nbkg_iteration)) / thread_ct))
        
        #setting up arrays for distances between data points and background MC and data points and signal MC
        signal_distances_GPU = np.zeros((nsig_iteration+nbkg_iteration)*nMC_sig, dtype = np.float32)
        background_distances_GPU = np.zeros((nsig_iteration+nbkg_iteration)*nMC_bkg, dtype = np.float32)
        
        #Calculating the distances between data points and MC
        signal_distances=distances_GPU[block_ct_sig, thread_ct](np.float32(data[0]), np.float32(data[1]), len(data[1]), np.float32(signal_points[0]), np.float32(signal_points[1]),len(signal_points[1]), signal_distances_GPU)
            
        background_distances=distances_GPU[block_ct_bkg, thread_ct](np.float32(data[0]), np.float32(data[1]), len(data[1]), np.float32(background_points[0]), np.float32(background_points[1]),len(background_points[1]), background_distances_GPU)
        
        #sorting the distances
        signal_prob=    sort_GPU((nsig_iteration+nbkg_iteration),nMC_sig,signal_distances_GPU,    nneigh)
        background_prob=sort_GPU((nsig_iteration+nbkg_iteration),nMC_bkg,background_distances_GPU,nneigh)
     

        # Generate MC bootstrap samples and calculate the probs for each
        signal_MC_bs = []
        background_MC_bs = []
      
        

        signal_prob_bs = []
        background_prob_bs = []
        for i in range(0,num_bootstrapping_samples):
            
            #Generating bootstrapping samples
            signal_MC_bs.append(bootstrapping(signal_points))
            background_MC_bs.append(bootstrapping(background_points))
            
            signal_distances_GPU = np.zeros((nsig_iteration+nbkg_iteration)*nMC_sig, dtype = np.float32)
            background_distances_GPU = np.zeros((nsig_iteration+nbkg_iteration)*nMC_bkg, dtype = np.float32)


            #calculating distances for each point
            signal_distances=distances_GPU[block_ct_sig, thread_ct](np.float32(data[0]), np.float32(data[1]), len(data[1]), np.float32(signal_MC_bs[i][0]), np.float32(signal_MC_bs[i][1]),len(signal_MC_bs[i][1]), signal_distances_GPU)
            
            background_distances=distances_GPU[block_ct_bkg, thread_ct](np.float32(data[0]), np.float32(data[1]), len(data[1]), np.float32(background_MC_bs[i][0]), np.float32(background_MC_bs[i][1]),len(background_MC_bs[i][1]), background_distances_GPU)
            
            #print signal_distances_GPU
            #print background_distances_GPU
            
            #sorting distances
            signal_prob_bs.append(sort_GPU((nsig_iteration+nbkg_iteration),nMC_sig,signal_distances_GPU,nneigh))
            background_prob_bs.append(sort_GPU((nsig_iteration+nbkg_iteration),nMC_bkg,background_distances_GPU,nneigh))
            
            #print signal_prob_bs[i]
            #print background_prob_bs[i]
           

        
        def tot_prob(frac,sig,bkg):
            tot_prob = frac*sig/nMC_sig + ((1-frac)*bkg/nMC_bkg)            
            return tot_prob
        
        def negative_log_likelihood(frac):
            
            # First, use the original MC/probs to calculate the NLL
            prob=tot_prob(frac,signal_prob,background_prob)
            #prob = np.float64(prob)
            nll =  -np.log(prob[prob>0]).sum()
            
            # Then add in the prob/NLLs for the bootstrap samples
            for i in range(0,num_bootstrapping_samples):
                prob = tot_prob(frac,signal_prob_bs[i],background_prob_bs[i])
                nll +=  -np.log(prob[prob>0]).sum()
            #print nll
            #print type(nll)
            return nll
        
        m1=Minuit(negative_log_likelihood,frac= 0.2,limit_frac=(0.001,1),error_frac=0.001,errordef =(num_bootstrapping_samples+1)*0.5,print_level=0)
        #m1.tol = num_bootstrapping_samples
        m1.migrad()
        #m1.hesse()

        if (m1.get_fmin().is_valid):
            param=m1.values
            err=m1.errors
            fit_frac.append(param["frac"])
            fit_frac_uncert.append(err["frac"])
            pull_frac=(frac_org-param["frac"])/err["frac"]
            pull_frac_list.append(pull_frac)
            
            output="%f %f %f %d %d %d %d %d %d %d %d\n" % (frac_org,param["frac"],err["frac"], nsig,nsig_iteration,nbkg,nbkg_iteration,nMC_sig,nMC_bkg,num_bootstrapping_samples,nneigh)
            #print output
            outfile.write(output)
    outfile.close()
    return pull_frac_list, frac, fit_frac, fit_frac_uncert,pull_iterations,outfile






